# **The Magpie algorithm**
> Copyright(C) Christopher Ehl - 17th of March 2023


The magpie algorithm is a destructive implementation of linear regression. Unlike Gradient Descent $^{[1]}$ the magpie algorithm uses N-dimensional vectors. The major selling point of the algorithm is the way it handles outliers. Outliers are discriminated and have very little effect on the rest of the calculations. Where other manual methods fail the magpie algorithm shines. With the good set of hyperparameters the algorithm can give a best fit line without fear of overfitting.

---

### Specifications

The runtime complexity of the algorithm is $ O(kN) $ for k iterations. The space complexity of the algorithm is $ O(N) $ *( for weight storage )*.

---

### Theory

Steps:

1. Read in data.
2. Split data into 2 parts.
3. Do the algorithm on both.
4. Extract line from the 2 remaining points.

Algorithm:

1. Sum up the vectors and weights.
2. Each vector will be updated by the formulae $ \sum_{i}^N \alpha \frac {v_j-v_i} {|v_j-v_i|} \frac {w_j} {w_i}$.
3. After each iteration, remove one of the points close to the other.

---

### Intuition

The algorithm is based on the law of gravitation $^{[2]}$. We treat every datapoint as a physical object and we simulate gravitational attraction between them. Since there is no velocity involved, they converge to one singularity.

## The Code

### Implementation

For the implementation I used julia $^{[3]}$, as it is a great language for scientific computing and has a very nice API to work with.

> It just works $^{TM}$

In [24]:
# Import linear algebra library
using LinearAlgebra

This implementation of ***magpie!*** is using pass by reference to make operations faster. The upgrade compared to the other implementations is that I used a very nice mathematical trick to get it run faster.

In [45]:
# Magpie O(N) implementation
function magpie!( V, W, α = 0.01 )
	@assert( length( V ) == length( W ) )

	# Number of data points
	N = length( V )
	# Sums of vectors and weights
	Σ_v, Σ_w = sum( V ), sum( W )
	
	for i = 1:N
		V[i] +=
			# learning-rate
			α *
			# difference-vector
			replace(
				normalize( Σ_v - V[i] * N ),
				NaN => 0.0
			) *
			# weight quotient
			( Σ_w / ( W[i] * N ) )
	end
end

magpie! (generic function with 2 methods)

This is an implementation of the deletion algorithm that uses a sliding window, and does not bother with distance calculation every point.

In [5]:
# Vector merge algorithm O(kn)
function delete_close!( V, W, β = 0.3, γ = 0.6, window_size = 1 )
	@assert( length( V ) == length( W ) )

	# Sliding window
	i = 1
	while i < length( V )
		# Flag deleted items		
		is_deleted = false
		# Slide the window
		for j = i+1 : min( i+1+window_size, length( V ) )
			# If vectors are too close
			if norm( V[i] - V[j] ) <= β
				# Delete them
				is_deleted = true
				W[j] = ( W[j] + W[i] ) * γ
				deleteat!( V, i )
				deleteat!( W, i )
				break
			end
		end
		# Continue
		if !is_deleted
			i += 1
		end
	end
end

delete_close! (generic function with 4 methods)

This is a wrapper for the algorithm that loops for N epochs.

In [54]:

# Magpie wrapper function
function iter_magpie!( epoch, V, W, α = 0.01, β = 0.3, γ = 0.5 )
	for i in 0:epoch
		magpie!( V, W, α )
		delete_close!( V, W, β, γ )
	end
end

iter_magpie! (generic function with 4 methods)

This is the final result run on some artificial data.

In [57]:
data = [
	[1.0,1.0],
	[2.0,1.0],
	[3.0,1.0],
	[4.0,1.0],
	[5.0,1.0]
]
weight = ones( Float32, length( data ) )
iter_magpie!( 10000, data, weight )
data

1-element Vector{Vector{Float64}}:
 [3.2800000000000367, 1.0]

---

### References

[1] Lemaréchal, C. (2012). "Cauchy and the Gradient Method" (PDF). Doc Math Extra: 251–254.

[2] Krebs, Robert E. (1999). Scientific Development and Misconceptions Through the Ages: A Reference Guide (illustrated ed.). Greenwood Publishing Group. p. 133. ISBN 978-0-313-30226-8.

[3] https://julialang.org/

[3] "2. Object-Oriented Programming - Beginning Julia Programming: For Engineers and Scientists [Book]". www.oreilly.com. Retrieved 26 January 2023.